# Uniswap V3 Data Notebook

In this notebook: 

- We use the TradingStrategy API to get data for 3 pairs on Uniswap V3
- This is done for 3 pairs: ETH/USDC, ETH/USDT, and USDT/USDC

The following code block shows an example of some the data from the api

In [5]:
import requests
import pandas as pd

pairs = {
    'eth_usdc': 'https://tradingstrategy.ai/api/candles?pair_id=2697765&exchange_type=uniswap_v3&time_bucket=4h',
    'usdt_usdc': 'https://tradingstrategy.ai/api/candles?pair_id=2711887&exchange_type=uniswap_v3&time_bucket=4h',
    'eth_usdt': 'https://tradingstrategy.ai/api/candles?pair_id=2697770&exchange_type=uniswap_v3&time_bucket=4h'

}

def get_candles(url: str) -> pd.DataFrame:
    x = requests.get(url)
    json = x.json()

    candles = pd.DataFrame.from_dict(json)
    candles.rename(columns = {'ts':'date','o':'open', 'h':'high','l':'low','c':'close','v':'volume'}, inplace = True)
    candles['timestamp'] = pd.to_datetime(candles['date'])
    candles = candles.set_index('date')
    return candles

pair_data = dict([ (k,get_candles(r)) for k,r in pairs.items()])

pair_data['usdt_usdc'].head()

,open,high,low,close,volume,bv,sv,xr,b,s,tc,timestamp
date,,,,,,,,,,,,
2023-03-25T20:00:00,1.001101,1.001101,1.000800,1.001101,4.783375e+07,0.0,4.783375e+07,1.0,None,None,None,2023-03-25 20:00:00
2023-03-26T00:00:00,1.001101,1.001101,1.001101,1.001101,1.371210e+06,0.0,1.371210e+06,1.0,None,None,None,2023-03-26 00:00:00
2023-03-26T04:00:00,1.001101,1.001101,1.001000,1.001000,3.509535e+06,0.0,3.509535e+06,1.0,None,None,None,2023-03-26 04:00:00
2023-03-26T08:00:00,1.001000,1.001101,1.000600,1.000900,3.847353e+07,0.0,3.847353e+07,1.0,None,None,None,2023-03-26 08:00:00
2023-03-26T12:00:00,1.000900,1.001101,1.000900,1.001000,2.600695e+07,0.0,2.600695e+07,1.0,None,None,None,2023-03-26 12:00:00


## Visual

Here we get the figures involving prices and volumes for each of the 3 pairs.

In [6]:
from tradingstrategy.charting.candle_chart import visualise_ohlcv
from tradingstrategy.charting.candle_chart import VolumeBarMode

def get_figure(candles: pd.DataFrame, chart_name: str):
    return visualise_ohlcv(
            candles=candles,
            height=600,
            theme="plotly_white",
            chart_name=chart_name,
            y_axis_name="Price",
            volume_axis_name="volume",
            volume_bar_mode=VolumeBarMode.separate,
            subplot_rel_size=0.5
        )

figures = []

for _chart_name, candles in pair_data.items():
    chart_name = _chart_name.replace('_','/').upper()
    figure = get_figure(candles, chart_name)     
    figures.append(figure)

## Show the figures

Now, we have a list of figures saved as a variable, and we can show each figure in the list. We display volume on a separate subplot since saving space is not a concern.

In [7]:

# To display the images

for figure in figures:
    figure.show()


## Save the figures

Alternatively, one could save them to your local harddrive. 

In [11]:
# To save the images

import plotly.io as pio

# # Save the figure to an image file
for figure in figures:
    title = figure.layout.title.text
    file_name = f"{title.replace('/', '_')}.png"
    pio.write_image(figure, file_name, format='png', width=800, height=600, scale=1)
    print(f"Image saved as {file_name}")

Image saved as ETH_USDC.png
Image saved as USDT_USDC.png
Image saved as ETH_USDT.png
